# Verify predictions

In [1]:
import pandas as pd
import os

In [2]:
s3_val_pred_ensemble_file_f1="s3://aegovan-data/pubmed_asbtract/predictions_valtest_ppimulticlass-bert-f1-2021-05-10-9_2021052215/val_multiclass.json.json"
s3_test_pred_ensemble_file_f1="s3://aegovan-data/pubmed_asbtract/predictions_valtest_ppimulticlass-bert-f1-2021-05-10-9_2021052215/test_multiclass.json.json"
s3_train_pred_ensemble_file_f1='s3://aegovan-data/pubmed_asbtract/predictions_valtest_ppimulticlass-bert-f1-2021-05-10-9_2021052910/train_multiclass.json.json'

s3_val_pred_ensemble_file_loss ="s3://aegovan-data/pubmed_asbtract/predictions_valtest_ppimulticlass-bert-2021-05-08-17-10_2021050918/val_multiclass.json.json"
s3_test_pred_ensemble_file_loss = "s3://aegovan-data/pubmed_asbtract/predictions_valtest_ppimulticlass-bert-2021-05-08-17-10_2021050918/test_multiclass.json.json"
#label_mapper = PpiMulticlassLabelMapper()

s3_prefix = "s3://aegovan-data/verify_ensemble_prediction_data"
s3_mainifests_prefix = "{}/mainifests".format(s3_prefix)


std_threshold = 0.15

In [3]:
s3_data_file = s3_test_pred_ensemble_file_f1

In [4]:
local_temp = "temp"
local_temp_pred_dir = os.path.join( local_temp, "pred_results")
local_temp_wk_dir = os.path.join( local_temp, "wk")

In [5]:
!rm -rf $local_temp
!mkdir -p $local_temp_pred_dir
!mkdir -p $local_temp_wk_dir

In [6]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def upload_file(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [7]:
%%time

download_file(s3_data_file, local_temp_pred_dir)

CPU times: user 83.1 ms, sys: 38.3 ms, total: 121 ms
Wall time: 1.95 s


In [8]:
data_full_df = pd.read_json(os.path.join(local_temp_pred_dir, s3_data_file.split("/")[-1]))

In [9]:
data_full_df.head(n=3)

,pubmedId,pubmedabstract,annotations,num_unique_gene_normalised_id,num_gene_normalised_id,normalised_abstract,normalised_abstract_annotations,participant1Id,participant2Id,gene_to_uniprot_map,...,phosphorylation,dephosphorylation,methylation,ubiquitination,acetylation,deubiquitination,prediction,confidence,confidence_std,raw_confidence
0,22621922,Cellular responses to DNA damage require the f...,"[{'start': '250', 'end': '275', 'name': 'vacci...",6,16,Cellular responses to DNA damage require the f...,"[{'charOffset': 250, 'len': 6, 'text': 'Q99986...",Q13315,Q12888,"{'7158': ['Q12888', 'A6NNK5'], '7443': ['Q9998...",...,0.002277,0.000474,0.000254,0.000344,0.000190,0.000224,other,0.996238,0.004872,"[0.9914163947105401, 0.9991012811660761, 0.998..."
1,22621922,Cellular responses to DNA damage require the f...,"[{'start': '250', 'end': '275', 'name': 'vacci...",6,16,Cellular responses to DNA damage require the f...,"[{'charOffset': 250, 'len': 6, 'text': 'Q99986...",Q12888,Q99986,"{'7158': ['Q12888', 'A6NNK5'], '7443': ['Q9998...",...,0.596406,0.026555,0.005901,0.057876,0.014610,0.008141,phosphorylation,0.596406,0.377836,"[0.8200772404670711, 0.9964775443077081, 0.630..."
2,19864627,NF-kappaB is a key activator of inflammatory a...,"[{'start': '0', 'end': '9', 'name': 'NF-kappaB...",5,15,P19838 is a key activator of inflammatory and ...,"[{'charOffset': 0, 'len': 6, 'text': 'P19838'}...",Q8WTS6,Q04206,"{'80854': ['Q8WTS6', 'B5MCZ8', 'D6RJA0'], '597...",...,0.008586,0.012597,0.377270,0.029645,0.101443,0.029655,other,0.440805,0.515117,"[0.004181528463959, 0.9565414190292351, 0.9814..."


In [10]:
def missing_uniprot(x):
    uniprots =[]
    for i in list(x["gene_to_uniprot_map"].values()):
        uniprots.extend(i)
        
    
    return x["participant1Id"] not in uniprots or x["participant2Id"] not in uniprots

def gene_id_map(x):
    gene_to_uniprot_map = x["gene_to_uniprot_map"]
    for n, uniprots in x["gene_to_uniprot_map"].items():
        if x["participant1Id"]  in uniprots :
            gene_to_uniprot_map[n]= x["participant1Id"]
        if x["participant2Id"]  in uniprots:
            gene_to_uniprot_map[n]= x["participant2Id"]
        
        
    return gene_to_uniprot_map
                                    


In [11]:

                                    
data_full_df["missing_uniprot"] = data_full_df.apply(missing_uniprot, axis=1)
missing_uniprots = data_full_df.query( "missing_uniprot == True").groupby("class")["class"].count()

assert len(missing_uniprots)==0

In [12]:
self_relation = data_full_df.query("participant1Id == participant2Id").groupby("class")["class"].count()
assert len(self_relation)==0

## Filter threshold

In [13]:
high_quality_preds_df = data_full_df.query(f"confidence_std <= {std_threshold} and prediction != 'other'")  
high_quality_preds_df.shape

(10, 25)

In [14]:
high_quality_preds_df

,pubmedId,pubmedabstract,annotations,num_unique_gene_normalised_id,num_gene_normalised_id,normalised_abstract,normalised_abstract_annotations,participant1Id,participant2Id,gene_to_uniprot_map,...,dephosphorylation,methylation,ubiquitination,acetylation,deubiquitination,prediction,confidence,confidence_std,raw_confidence,missing_uniprot
6,20856200,The PI3K/AKT signaling pathway is aberrant in ...,"[{'start': '9', 'end': '12', 'name': 'AKT', 't...",2,15,The PI3K/P31749 signaling pathway is aberrant ...,"[{'charOffset': 9, 'len': 6, 'text': 'P31749'}...",P31749,P08670,"{'7431': ['P08670', 'V9HWE1'], '207': ['P31749...",...,0.002388,0.006001,0.014256,0.003362,0.007223,phosphorylation,0.957124,0.048810,"[0.9740308523178101, 0.9722574353218071, 0.959...",False
18,9012831,Bruton's tyrosine kinase (Btk) is essential fo...,"[{'start': '0', 'end': '24', 'name': 'Bruton's...",4,23,Q06187 (Q06187) is essential for B cell activa...,"[{'charOffset': 0, 'len': 6, 'text': 'Q06187'}...",P78347,Q06187,"{'2969': ['P78347', 'X5DR09', 'X5D939', 'Q499G...",...,0.024810,0.001643,0.002249,0.003572,0.001567,phosphorylation,0.929067,0.121474,"[0.9939718842506401, 0.995732367038726, 0.8928...",False
21,22726438,Receptor tyrosine kinase activity is known to ...,"[{'start': '331', 'end': '336', 'name': 'FGFR2...",2,7,Receptor tyrosine kinase activity is known to ...,"[{'charOffset': 331, 'len': 6, 'text': 'P21802...",P21802,P62993,"{'2885': ['P62993', 'B0LPF3'], '2263': ['P2180...",...,0.004533,0.002220,0.003528,0.002576,0.002809,phosphorylation,0.940786,0.071991,"[0.829028844833374, 0.9946938157081601, 0.9794...",False
31,19424295,Proper regulation of the cAMP-dependent protei...,"[{'start': '165', 'end': '170', 'name': 'human...",4,8,Proper regulation of the cAMP-dependent protei...,"[{'charOffset': 250, 'len': 6, 'text': 'P10644...",P23528,P53667,"{'1072': ['P23528', 'V9HWI5'], '5573': ['P1064...",...,0.003005,0.002878,0.006154,0.003950,0.004398,phosphorylation,0.963665,0.083319,"[0.989717006683349, 0.9877800941467281, 0.9939...",False
46,25605758,Leucine-rich repeat kinase 2 (LRRK2) has been ...,"[{'start': '0', 'end': '28', 'name': 'Leucine-...",4,22,Q5S007 (Q5S007) has been identified as a causa...,"[{'charOffset': 0, 'len': 6, 'text': 'Q5S007'}...",Q5S007,P61020,"{'5868': ['P20339', 'A0A024R2K1'], '1956': ['P...",...,0.001701,0.001625,0.001346,0.001016,0.001280,phosphorylation,0.988185,0.006639,"[0.979663550853729, 0.9917544126510621, 0.9900...",False
51,11154276,The Akt family of serine/threonine-directed ki...,"[{'start': '4', 'end': '7', 'name': 'Akt', 'ty...",3,21,The P31749 family of serine/threonine-directed...,"[{'charOffset': 4, 'len': 6, 'text': 'P31749'}...",P31749,Q99683,"{'4217': ['Q99683'], '207': ['P31749', 'B0LPE5...",...,0.003736,0.001073,0.001400,0.001292,0.001060,phosphorylation,0.954891,0.100438,"[0.687737703323364, 0.9960561990737911, 0.9875...",False
53,21920476,Protein phosphorylation by kinases plays a cen...,"[{'start': '548', 'end': '553', 'name': 'human...",3,12,Protein phosphorylation by kinases plays a cen...,"[{'charOffset': 580, 'len': 6, 'text': 'Q99986...",P38432,Q99986,"{'7443': ['Q99986'], '8161': ['P38432'], '7444...",...,0.002694,0.002624,0.007484,0.002612,0.003666,phosphorylation,0.975895,0.033522,"[0.9866229891777031, 0.99492996931076, 0.99034...",False
209,15862297,By GST pull downs and co-immunoprecipitation a...,"[{'start': '80', 'end': '84', 'name': 'Chk2', ...",3,18,By GST pull downs and co-immunoprecipitation a...,"[{'charOffset': 80, 'len': 6, 'text': 'O96017'...",P04637,Q00987,"{'4193': ['Q00987', 'A7UKX8', 'G3XA89', 'A7UKX...",...,0.003202,0.001213,0.001555,0.001589,0.001150,phosphorylation,0.962055,0.049956,"[0.9909093976020811, 0.9949020147323601, 0.985...",False
344,21986944,"Male germ cell-associated kinase (MAK), a dire...","[{'start': '0', 'end': '32', 'name': 'Male ger...",5,21,"P20794 (P20794), a direct transcriptional targ...","[{'charOffset': 0, 'len': 6, 'text': 'P20794'}...",P12830,P20794,"

In [27]:
high_quality_preds_df[["pubmedId","participant1Name","participant1Id" , "participant2Name", "participant2Id", "prediction", 'class']]

,pubmedId,participant1Name,participant1Id,participant2Name,participant2Id,prediction,class
6,20856200,AKT,P31749,vimentin,P08670,phosphorylation,phosphorylation
18,9012831,BAP-135,P78347,Bruton's tyrosine kinase,Q06187,phosphorylation,phosphorylation
21,22726438,FGFR2,P21802,Grb2,P62993,phosphorylation,phosphorylation
31,19424295,cofilin,P23528,Limk1,P53667,phosphorylation,phosphorylation
46,25605758,Leucine-rich repeat kinase 2,Q5S007,Rab5b,P61020,phosphorylation,phosphorylation
51,11154276,Akt,P31749,apoptosis signal-regulating kinase 1,Q99683,phosphorylation,phosphorylation
53,21920476,coilin,P38432,VRK1,Q99986,phosphorylation,phosphorylation
209,15862297,p53,P04637,HDM2,Q00987,phosphorylation,other
344,21986944,CDH1,P12830,MAK,P20794,phosphorylation,other
350,21887822,Hsp70,P34932,Plk1,P53350,phosphorylation,other


In [16]:
high_quality_preds_df.head()

,pubmedId,pubmedabstract,annotations,num_unique_gene_normalised_id,num_gene_normalised_id,normalised_abstract,normalised_abstract_annotations,participant1Id,participant2Id,gene_to_uniprot_map,...,dephosphorylation,methylation,ubiquitination,acetylation,deubiquitination,prediction,confidence,confidence_std,raw_confidence,missing_uniprot
6,20856200,The PI3K/AKT signaling pathway is aberrant in ...,"[{'start': '9', 'end': '12', 'name': 'AKT', 't...",2,15,The PI3K/P31749 signaling pathway is aberrant ...,"[{'charOffset': 9, 'len': 6, 'text': 'P31749'}...",P31749,P08670,"{'7431': ['P08670', 'V9HWE1'], '207': ['P31749...",...,0.002388,0.006001,0.014256,0.003362,0.007223,phosphorylation,0.957124,0.048810,"[0.9740308523178101, 0.9722574353218071, 0.959...",False
18,9012831,Bruton's tyrosine kinase (Btk) is essential fo...,"[{'start': '0', 'end': '24', 'name': 'Bruton's...",4,23,Q06187 (Q06187) is essential for B cell activa...,"[{'charOffset': 0, 'len': 6, 'text': 'Q06187'}...",P78347,Q06187,"{'2969': ['P78347', 'X5DR09', 'X5D939', 'Q499G...",...,0.024810,0.001643,0.002249,0.003572,0.001567,phosphorylation,0.929067,0.121474,"[0.9939718842506401, 0.995732367038726, 0.8928...",False
21,22726438,Receptor tyrosine kinase activity is known to ...,"[{'start': '331', 'end': '336', 'name': 'FGFR2...",2,7,Receptor tyrosine kinase activity is known to ...,"[{'charOffset': 331, 'len': 6, 'text': 'P21802...",P21802,P62993,"{'2885': ['P62993', 'B0LPF3'], '2263': ['P2180...",...,0.004533,0.002220,0.003528,0.002576,0.002809,phosphorylation,0.940786,0.071991,"[0.829028844833374, 0.9946938157081601, 0.9794...",False
31,19424295,Proper regulation of the cAMP-dependent protei...,"[{'start': '165', 'end': '170', 'name': 'human...",4,8,Proper regulation of the cAMP-dependent protei...,"[{'charOffset': 250, 'len': 6, 'text': 'P10644...",P23528,P53667,"{'1072': ['P23528', 'V9HWI5'], '5573': ['P1064...",...,0.003005,0.002878,0.006154,0.003950,0.004398,phosphorylation,0.963665,0.083319,"[0.989717006683349, 0.9877800941467281, 0.9939...",False
46,25605758,Leucine-rich repeat kinase 2 (LRRK2) has been ...,"[{'start': '0', 'end': '28', 'name': 'Leucine-...",4,22,Q5S007 (Q5S007) has been identified as a causa...,"[{'charOffset': 0, 'len': 6, 'text': 'Q5S007'}...",Q5S007,P61020,"{'5868': ['P20339', 'A0A024R2K1'], '1956': ['P...",...,0.001701,0.001625,0.001346,0.001016,0.001280,phosphorylation,0.988185,0.006639,"[0.979663550853729, 0.9917544126510621, 0.9900...",False


In [17]:
import json
import tempfile
def create_manifest_file(df, s3_output_manifest_file):
    items = df.to_dict(orient='records' )
    temp_file = os.path.join( tempfile.mkdtemp(), s3_output_manifest_file.split("/")[-1])
    with open(temp_file , "w") as f:
        for item in items:
            # Write without new lines
            item_m  = {}
            item_m["source"] = json.dumps(item)
            f.write(json.dumps(item_m).replace("\n", "\t"))
            f.write("\n")
    upload_file(temp_file, s3_output_manifest_file)

In [18]:
 "{}/".format(s3_mainifests_prefix.rstrip("/"))

's3://aegovan-data/verify_ensemble_prediction_data/mainifests/'

In [19]:
manifest_file = "{}/{}".format( s3_mainifests_prefix.rstrip("/"), s3_data_file.split("/")[-1])
create_manifest_file( high_quality_preds_df, manifest_file)

s3_manifests = [manifest_file]

In [20]:
s3_manifests

['s3://aegovan-data/verify_ensemble_prediction_data/mainifests/test_multiclass.json.json']

### Create sagemaker ground truth labelling job

In [21]:
import boto3
import sagemaker

In [22]:
from datetime import datetime

def create_groundtruth_labelling_job(s3_manifest, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_name, label_attribute_name="prediction", workforce_type= "private-crowd" ):
    client = boto3.client('sagemaker')
    
    sagemaker_session = sagemaker.Session()
    account_id =  boto3.client('sts').get_caller_identity().get('Account')
    region = boto3.session.Session().region_name
    
    workforce_arn = "arn:aws:sagemaker:{}:{}:workteam/{}/{}".format(region, account_id, workforce_type, workforce_name)
    role_arn = "arn:aws:iam::{}:role/{}".format( account_id,  role)
    pre_lambda_arn = "arn:aws:lambda:{}:{}:function:{}".format(region, account_id,  pre_lambda)
    post_lambda_arn = "arn:aws:lambda:{}:{}:function:{}".format(region, account_id,  post_lambda)
    
    num_workers_per_object = 1
    task_time_limit_sec = 60  * 60 * 5
    task_availablity_sec =60  * 60 * 24 * 10
    
    job = client.create_labeling_job(LabelingJobName=job_name
                                    ,LabelAttributeName = label_attribute_name
                                    ,InputConfig = {
                                        "DataSource": {
                                            'S3DataSource': {
                                            'ManifestS3Uri': s3_manifest
                                            }
                                        }
                                        
                                    }
                                  ,OutputConfig={
                                        'S3OutputPath': s3_gt_output
                                    }

                                  , RoleArn = role_arn
                                  , HumanTaskConfig={
                                    'WorkteamArn': workforce_arn,
                                    'UiConfig': {
                                        'UiTemplateS3Uri': s3_template
                                    },
                                    'PreHumanTaskLambdaArn': pre_lambda_arn,
                                    'TaskKeywords': [
                                        'PPI',
                                    ],
                                    'TaskTitle': 'Verify PPI extraction for protein {}'.format(s3_manifest.split("/")[-1]),
                                    'TaskDescription': 'Verifies PPi extraction',
                                    'NumberOfHumanWorkersPerDataObject': num_workers_per_object,
                                    'TaskTimeLimitInSeconds': task_time_limit_sec,
                                    'TaskAvailabilityLifetimeInSeconds': task_availablity_sec,
                                    'MaxConcurrentTaskCount': 10,
                                    'AnnotationConsolidationConfig': {
                                        'AnnotationConsolidationLambdaArn': post_lambda_arn
                                    }
                                }
                            )
    
    return job
    
    

def create_groundtruth_labelling_multiple_jobs(lst_s3_manifests, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_prefix ="ppi", label_attribute_name="class"):
    job_prefix = "{}-{}".format(job_prefix , datetime.now().strftime("%Y%m%d%H%M%S"))
    for s3_manifest in lst_s3_manifests:
        job_name = "{}-{}".format( job_prefix, s3_manifest.split("/")[-1].split("_")[-1].split(".")[0])
        create_groundtruth_labelling_job(s3_manifest, s3_gt_output, s3_template, pre_lambda, post_lambda, role, workforce_name, job_name)

In [23]:
import urllib.request

def download_template(template_url):
    with urllib.request.urlopen(template_url) as f:
        html = f.read().decode('utf-8')

    with open("template.html", "w") as f:
        f.write(html)
    
download_template('http://raw.githubusercontent.com/elangovana/ppi-sagemaker-groundtruth-verification/main/src/template/template.html')

In [24]:
s3_prefix.rstrip("/")

's3://aegovan-data/verify_ensemble_prediction_data'

In [25]:
role_name = "service-role/AmazonSageMaker-ExecutionRole-20210104T161547"
pre_lambda="Sagemaker-ppipreprocessing"
post_lambda="sagemaker-ppipostprocessing"
s3_gt_output = "{}/gt_output/".format(s3_prefix.rstrip("/"))
workforce_name = "ppi-team"
s3_template_file = "{}/template.html".format(s3_prefix.rstrip("/"))

upload_file("template.html", s3_template_file )
create_groundtruth_labelling_multiple_jobs (s3_manifests,
                                            s3_gt_output, 
                                            s3_template_file,
                                            pre_lambda, 
                                            post_lambda, 
                                            role_name,
                                            workforce_name)